In [ ]:
"""test marker representation on sample sequences"""

In [18]:
import os
import pickle
import random
import multiprocessing
import numpy as np
from itertools import product

In [2]:
# define data paths
data_dir = '/ccb/salz4-4/markus/markraken/data'

HPC_pickle_path = os.path.join(data_dir, 'DB_HPC.pkl')

In [3]:
# load data
with open(HPC_pickle_path, 'rb') as f:
    HPC_list = pickle.load(f)

In [63]:
n_markers = 8000 # number of markers to use for representation of sequence as markers, Shasta default = 8000
marker_length = 10 # length of marker sequence, Shasta default = 10

# randomly select marker set without replacement, surprisingly complex to vectorize, works fast enough like this
random.seed(2019)
marker_set = set()
while len(marker_set) < n_markers:
    marker = [random.sample('ATCG', 1)[0]]
    for i in range(marker_length):
        nucs = {'A', 'T', 'C', 'G'}
        nucs.remove(marker[-1])
        marker.append(random.sample(nucs, 1)[0])
    marker_set.add(''.join(marker))

marker_list = list(marker_set)

In [ ]:
def markerize(HPC_seq):
    
    return marker_seq